In [1]:
import torch
import torchvision
from torchvision import transforms, datasets, models
from torchvision.utils import save_image, make_grid

from torchviz import make_dot

import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import math


import os 
import glob
import time
import cv2

from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt

import imageio

import enum

In [2]:
torch.cuda.is_available()
device = torch.device("cuda")

In [3]:
class Generator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Generator, self).__init__()
        
        #reduce to features
        self.c0 = nn.Conv2d(in_channels, 64, 4, stride=2, padding=1)
        self.c1 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.c2 = nn.Conv2d(128, 256, 4, stride=2, padding=1)
        self.c3 = nn.Conv2d(256, 512, 4, stride=2, padding=1)
        self.c4 = nn.Conv2d(512, 512, 4, stride=2, padding=1)
        self.c5 = nn.Conv2d(512, 512, 4, stride=2, padding=1)
        self.c6 = nn.Conv2d(512, 512, 4, stride=2, padding=1)
        self.c7 = nn.Conv2d(512, 512, 4, stride=2, padding=1)
        
        #upsample to image
        self.d7 = nn.ConvTranspose2d(512, 512, 4, stride=2, padding=1)
        self.d6 = nn.ConvTranspose2d(1024, 512, 4, stride=2, padding=1)
        self.d5 = nn.ConvTranspose2d(1024, 512, 4, stride=2, padding=1)
        self.d4 = nn.ConvTranspose2d(1024, 512, 4, stride=2, padding=1)
        self.d3 = nn.ConvTranspose2d(1024, 256, 4, stride=2, padding=1)
        self.d2 = nn.ConvTranspose2d(512, 128, 4, stride=2, padding=1)
        self.d1 = nn.ConvTranspose2d(256, 64, 4, stride=2, padding=1)
        self.d0 = nn.ConvTranspose2d(128, out_channels, 4, stride=2, padding=1)
       
        self.bnc1 = nn.BatchNorm2d(128)
        self.bnc2 = nn.BatchNorm2d(256)
        self.bnc3 = nn.BatchNorm2d(512)
        self.bnc4 = nn.BatchNorm2d(512)
        self.bnc5 = nn.BatchNorm2d(512)
        self.bnc6 = nn.BatchNorm2d(512)

        self.bnd7 = nn.BatchNorm2d(512)
        self.bnd6 = nn.BatchNorm2d(512)
        self.bnd5 = nn.BatchNorm2d(512)
        self.bnd4 = nn.BatchNorm2d(512)
        self.bnd3 = nn.BatchNorm2d(256)
        self.bnd2 = nn.BatchNorm2d(128)
        self.bnd1 = nn.BatchNorm2d(64)
        
    def forward(self, x):           
        en0 = self.c0(x)
        en1 = self.bnc1(self.c1(F.leaky_relu(en0, negative_slope=0.2)))
        en2 = self.bnc2(self.c2(F.leaky_relu(en1, negative_slope=0.2)))
        en3 = self.bnc3(self.c3(F.leaky_relu(en2, negative_slope=0.2)))
        en4 = self.bnc4(self.c4(F.leaky_relu(en3, negative_slope=0.2)))
        en5 = self.bnc5(self.c5(F.leaky_relu(en4, negative_slope=0.2)))
        en6 = self.bnc6(self.c6(F.leaky_relu(en5, negative_slope=0.2)))
        en7 = self.c7(F.leaky_relu(en6, negative_slope=0.2))

        de7 = self.bnd7(self.d7(F.relu(en7)))
        de6 = F.dropout(self.bnd6(self.d6(F.relu(torch.cat((en6, de7),1)))))
        de5 = F.dropout(self.bnd5(self.d5(F.relu(torch.cat((en5, de6),1)))))

        de4 = F.dropout(self.bnd4(self.d4(F.relu(torch.cat((en4, de5),1)))))
        de3 = self.bnd3(self.d3(F.relu(torch.cat((en3, de4),1))))
        de2 = self.bnd2(self.d2(F.relu(torch.cat((en2, de3),1))))
        de1 = self.bnd1(self.d1(F.relu(torch.cat((en1, de2),1))))

        de0 = torch.tanh(self.d0(F.relu(torch.cat((en0, de1),1))))       

        return de0


In [13]:


images = []
j = 0
for i in glob.glob("./generated_images/*"):    
    img = imageio.imread(i)
    #cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
    img = cv2.putText(img, str(j), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)
    images.append(img)
    j+=1
    

imageio.mimwrite("epochs.gif", images, fps = 50, loop = 0)    

RuntimeError: Zero images were written.

In [10]:
netG = Generator(1,1).to(device)
netG.load_state_dict(torch.load('./epochModelParamData/G105.pth'))
netG.eval()

pug = cv2.imread('./ThePugTest.png', cv2.IMREAD_GRAYSCALE)
#pug = y
pug = pug.astype(np.float32)
pug = pug/127.5 - 1 
pug = torch.tensor(pug).to(device)
pug = pug.unsqueeze(0).unsqueeze(0)
print(pug.shape)

pug_plate_of_truth = netG(pug)

pug_plate_of_truth = pug_plate_of_truth.squeeze(0).squeeze(0)
pug = pug.squeeze(0).squeeze(0)
pug_plate = torch.cat((pug,pug_plate_of_truth),1)
pug_plate = pug_plate.cpu().detach().numpy()

pug_plate = (pug_plate + 1)/2 *255
pug_plate = pug_plate.astype(np.uint8)
%matplotlib qt
plt.imshow(pug_plate, cmap="gray")
imageio.imwrite('pug_plate.png', pug_plate)

torch.Size([1, 1, 256, 512])


In [8]:
%matplotlib qt
y  = np.zeros((256, 512)).astype(np.uint8) + 255
start = (106,156//2) 
end = (406, 156//2 + 100)
start2 = (106+3,156//2+3) 
end2 = (406-3, 156//2 + 100 -3)
y = cv2.rectangle(y, start, end, (0,0,0), -1)
y = cv2.rectangle(y, start2, end2, (255,255,255), 2)
y = cv2.putText(y, "UP 32 FV 7298", (132, 139), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
#y = cv2.blur(y, (5,5))

pmatrix = np.array(([0.8647, 0.01, 19.4354], [0.1762, 0.8919, 25.2840], [0,0,1]))

#pmatrix = np.array(([1,0,0], [0,1,0], [0,0,1])).astype(np.float32)

y = cv2.warpPerspective(y, pmatrix, dsize = (512, 256), borderValue = np.ones(3)*255)


plt.imshow(y, cmap = "gray")




imageio.imwrite('./y.png', y)


In [35]:
np.random.rand(3)

array([0.83398015, 0.57532914, 0.16970007])